In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")

In [3]:
train.shape, test.shape


((1460, 81), (1459, 80))

In [4]:
train.isna().sum().sort_values()[-19:-1]

Electrical         1
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
BsmtFinType2      38
BsmtExposure      38
GarageQual        81
GarageFinish      81
GarageYrBlt       81
GarageType        81
GarageCond        81
LotFrontage      259
FireplaceQu      690
Fence           1179
Alley           1369
MiscFeature     1406
dtype: int64

In [5]:
test.isna().sum().sort_values()[-33:-1]

GarageCars         1
GarageArea         1
KitchenQual        1
Exterior1st        1
SaleType           1
TotalBsmtSF        1
BsmtUnfSF          1
Exterior2nd        1
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFullBath       2
Functional         2
Utilities          2
BsmtHalfBath       2
MSZoning           4
MasVnrArea        15
MasVnrType        16
BsmtFinType2      42
BsmtFinType1      42
BsmtExposure      44
BsmtQual          44
BsmtCond          45
GarageType        76
GarageFinish      78
GarageYrBlt       78
GarageQual        78
GarageCond        78
LotFrontage      227
FireplaceQu      730
Fence           1169
Alley           1352
MiscFeature     1408
dtype: int64

In [6]:
#Filling with "NA" string
for col in ['Alley','FireplaceQu','Fence','MiscFeature','PoolQC','GarageQual','GarageFinish','GarageYrBlt','GarageType','GarageCond','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure']:
    train[col].fillna('NA', inplace=True)
    test[col].fillna('NA', inplace=True)

In [7]:
train["LotFrontage"].value_counts().to_frame().index[0]

60.0

In [8]:
#Fill the most commonly occuring number i.e mode for the strings
train['LotFrontage'].fillna(train["LotFrontage"].value_counts().to_frame().index[0], inplace=True)
test['LotFrontage'].fillna(test["LotFrontage"].value_counts().to_frame().index[0], inplace=True)
train['Electrical'].fillna('SBrkr',inplace=True)


In [9]:
for col in []:
    train[col].fillna('NA',inplace=True)
    test[col].fillna('NA',inplace=True)

In [10]:
categorical = ['KitchenQual','Exterior1st','SaleType','Exterior2nd','Functional','Utilities','MSZoning']
for i in categorical:
    train[i].fillna(train[i].value_counts().to_frame().index[0], inplace=True)
    test[i].fillna(test[i].value_counts().to_frame().index[0], inplace=True)

In [11]:
print(train.isna().sum().sort_values()[-15:-1])
print(test.isna().sum().sort_values()[-15:-1])

BsmtFinType1    0
BsmtExposure    0
BsmtCond        0
BsmtQual        0
Foundation      0
ExterCond       0
ExterQual       0
Exterior2nd     0
Exterior1st     0
RoofMatl        0
RoofStyle       0
Heating         0
SalePrice       0
MasVnrArea      8
dtype: int64
BsmtCond         0
YearRemodAdd     0
ExterCond        0
Foundation       0
BsmtQual         0
TotalBsmtSF      1
BsmtFinSF1       1
BsmtUnfSF        1
GarageArea       1
GarageCars       1
BsmtFinSF2       1
BsmtHalfBath     2
BsmtFullBath     2
MasVnrArea      15
dtype: int64
